In [ ]:
import panel as pn
import numpy as np
import xarray as xr
import holoviews as hv
from holoviews import streams
import hvplot.xarray  # noqa: F401 – registers hvplot accessor
import hvplot.pandas



In [ ]:
def calculate_phasor_transform(ds: xr.Dataset) -> xr.Dataset:
    fft_values = xr.apply_ufunc(
        np.fft.fft,
        ds.intensity,
        input_core_dims=[["wavelength"]],
        output_core_dims=[["harmonic_bin"]],
    )
    dc_component = fft_values.isel(harmonic_bin=0).real
    h = int(ds.attrs.get("harmonic", 1))
    harmonic_data = fft_values.isel(harmonic_bin=h)
    ds["G"] = harmonic_data.real / dc_component
    ds["S"] = harmonic_data.imag / dc_component
    return ds


def add_spectral_reference(ds: xr.Dataset, n_ref_points: int = 360) -> xr.Dataset:
    harmonic = int(ds.attrs.get("harmonic", 1))
    w_min = ds["wavelength"].min().item()
    w_max = ds["wavelength"].max().item()
    w_ref = np.linspace(w_min, w_max, n_ref_points)
    indices = np.arange(n_ref_points)
    phase = -2 * np.pi * harmonic * indices / n_ref_points
    ds = ds.assign_coords(wavelength_ref=w_ref)
    ds["G_ref"] = (("wavelength_ref",), np.cos(phase))
    ds["S_ref"] = (("wavelength_ref",), np.sin(phase))
    return ds


def make_gaussian(wavelengths: np.ndarray, mean: float, std: float) -> np.ndarray:
    return np.exp(-0.5 * ((wavelengths - mean) / std) ** 2)


def make_dataset(
    wavelengths: np.ndarray, intensities: np.ndarray, harmonic_n: int
) -> xr.Dataset:
    """Build a clean xr.Dataset from a (samples × wavelength) array."""
    n_samples = intensities.shape[0]
    ds = xr.Dataset(
        data_vars={
            "intensity": (["sample", "wavelength"], intensities),
            "G": (["sample"], np.full(n_samples, np.nan)),
            "S": (["sample"], np.full(n_samples, np.nan)),
        },
        coords={
            "sample": np.arange(n_samples),
            "wavelength": wavelengths,
        },
        attrs={"harmonic": harmonic_n},
    )
    ds.wavelength.attrs = {"units": "nm"}
    ds.G.attrs = {"long_name": "Phasor G (real)"}
    ds.S.attrs = {"long_name": "Phasor S (imaginary)"}
    return ds


def get_wavelengths(start: float, step: float, end: float) -> np.ndarray:
    return np.arange(start, end + step / 2, step)


# ── Plot helpers ─────────────────────────────────────────────────────────────


def create_phasor_plot(ds: xr.Dataset, sample_cmap: str = "Set1"):
    df_samples = ds[["G", "S"]].to_dataframe().reset_index()
    df_ref = ds[["G_ref", "S_ref"]].to_dataframe().reset_index()

    ref_chart = hv.Points(
        hv.Dataset(df_ref), kdims=["G_ref", "S_ref"], vdims=["wavelength_ref"]
    ).opts(
        color="wavelength_ref",
        cmap="rainbow4",
        size=4,
        colorbar=False,
        tools=["hover"],
        clabel="Wavelength (nm)",
    )

    single_sample = len(df_samples) == 1
    sample_opts = dict(
        marker="circle",
        size=12,
        tools=["hover", "box_select", "lasso_select", "tap"],
        show_legend=False,
    )
    if single_sample:
        sample_opts["color"] = "steelblue"
    else:
        sample_opts["color"] = "sample"
        sample_opts["cmap"] = sample_cmap

    sample_chart = hv.Points(
        hv.Dataset(df_samples), kdims=["G", "S"], vdims=["sample"]
    ).opts(**sample_opts)

    phasor_plot = (ref_chart * sample_chart).opts(
        hv.opts.Points(
            frame_width=500,
            frame_height=500,
            padding=0.1,
            xlabel="G",
            ylabel="S",
            show_grid=True,
            title="Phasor Plot",
        )
    )
    return phasor_plot, sample_chart


def build_spectrum_dmap(
    ds: xr.Dataset,
    sample_plot: hv.Points,
    show_individual: bool = False,
    cmap: str = "viridis",
) -> hv.DynamicMap:
    """Return a DynamicMap wired to *sample_plot*'s Selection1D stream.

    When show_individual is True the lines are coloured with *cmap* mapped
    over the sample index, matching the phasor dot colours exactly.
    """
    selection = streams.Selection1D(source=sample_plot)
    single_sample_ds = len(ds.sample) == 1

    def select_spectrum(index):
        if not index:
            selected = ds.intensity
            title = "All Samples"
        else:
            selected = ds.intensity.isel(sample=index)
            title = f"{len(index)} Selected"

        n = len(selected.sample)

        if show_individual and n <= 100:
            if single_sample_ds or n == 1:
                # Single point → use the same fixed colour as the phasor dot
                plot = selected.mean("sample", keep_attrs=True).hvplot.line(
                    x="wavelength",
                    y="intensity",
                    color="steelblue",
                    title=title,
                    legend=False,
                )
                plot = hv.NdOverlay({"0": plot}, kdims="sample")
            else:
                # Multi-sample → colour by sample index with the same cmap
                # as the phasor plot so colours correspond 1-to-1
                plot = selected.hvplot.line(
                    x="wavelength",
                    y="intensity",
                    by="sample",
                    cmap=cmap,
                    clim=(float(ds.sample.min()), float(ds.sample.max())),
                    alpha=0.7,
                    title=title,
                    legend=False,
                )
        else:
            suffix = " [safe mode: >100]" if show_individual and n > 100 else ""
            avg_curve = selected.mean("sample", keep_attrs=True).hvplot.line(
                x="wavelength",
                y="intensity",
                title=f"Average Spectrum ({title}){suffix}",
                color="black",
            )
            plot = hv.NdOverlay({"Average": avg_curve}, kdims="sample")

        return plot.opts(width=500, height=500)

    return hv.DynamicMap(select_spectrum, streams=[selection])

In [ ]:
wavelengths = np.linspace(400, 700, 256)
sample_indices = np.arange(300)

spectra_xr_ds = xr.Dataset(
    data_vars={
        # Intensity is 2D: (sample x wavelength)
        "intensity": (
            ["sample", "wavelength"],
            # np.full((len(sample_indices), len(wavelengths)), np.nan),
            np.random.rand(len(sample_indices), len(wavelengths)),
        ),
        # Phasors are 1D: (sample)
        "G": (["sample"], np.full(len(sample_indices), np.nan)),
        "S": (["sample"], np.full(len(sample_indices), np.nan)),
    },
    coords={
        "sample": sample_indices,
        "wavelength": wavelengths,
    },
)

spectra_xr_ds.attrs["harmonic"] = 1
# Add units immediately so you don't forget
spectra_xr_ds.wavelength.attrs = {"units": "nm"}

spectra_xr_ds.G.attrs = {"long_name": "Phasor G (real)"}
spectra_xr_ds.S.attrs = {"long_name": "Phasor S (imaginary)"}

ds = spectra_xr_ds
ds = calculate_phasor_transform(ds)
ds = add_spectral_reference(ds)


df_samples = ds[["G", "S"]].to_dataframe().reset_index()
df_ref = ds[["G_ref", "S_ref"]].to_dataframe().reset_index()

In [ ]:

# ref_chart = hv.Points(
#     hv.Dataset(df_ref), kdims=["G_ref", "S_ref"], vdims=["wavelength_ref"]
# ).opts(
#     color="wavelength_ref",
#     size=4,
#     colorbar=False,
#     tools=["hover"],
#     clabel="Wavelength (nm)",
# )

# ref_chart

In [ ]:
hvplot.extension("bokeh")



ref_chart = df_ref.hvplot.points(x="G_ref", y="S_ref", color="wavelength_ref", cmap="spectral_r",
        size=12, colorbar=False, tools=["hover"])

sample_chart = df_samples.hvplot.points(x="G", y="S", color="sample", cmap=color_mapping,
        size=12, colorbar=False, tools=["hover", "box_select", "lasso_select", "tap"])



phasor_plot = (ref_chart * sample_chart).opts(
hv.opts.Points(
        frame_width=200,
        frame_height=200,
        size=12,
        padding=0.1,
        xlabel="G",
        ylabel="S",
        show_grid=True,
        title="Phasor Plot",
)
)

selection = streams.Selection1D(source=sample_chart)




def select_spectrum(index):
        
        if not index:
                plot = ds.intensity.hvplot.line(
                x="wavelength",
                y="intensity",
                by="sample",
                cmap=color_mapping,
                legend=False,
                )
                
        else:
        
                plot = ds.intensity.hvplot.line(
                        x="wavelength",
                        y="intensity",
                        by="sample",
                        cmap=color_mapping,
                        legend=False,
                        ).select(sample=index).collate()
        

        return plot.opts(width=200, height=200)

phasor_plot + hv.DynamicMap(select_spectrum, streams=[selection])

In [ ]:
print(ds.intensity.to_dataframe().reset_index())

In [ ]:
hv.dim("sample").categorize(color_mapping)
color_mapping

In [ ]:
tuple(color_mapping.values())

In [ ]:
hv.extension('bokeh')

from itertools import cycle
import colorcet
color_mapping = dict(zip(df_samples["sample"].to_list(), cycle(colorcet.b_glasbey_hv)))

# 1. Create the HoloViews Dataset
# We define wavelength and sample as key dimensions (kdims)
hv_ds = hv.Dataset(ds.intensity.to_dataframe().reset_index(), kdims=['wavelength', 'sample'], vdims='intensity')

# 2. Convert to Curves and overlay by 'sample'
# This creates an NdOverlay object
overlay = hv_ds.to(hv.Curve, 'wavelength', 'intensity').overlay('sample')

# 3. Apply styling and a specific colormap
# We use hv.Cycle() to map the colormap to the categorical 'sample' dimension
plot = overlay.opts(
    hv.opts.Curve(
        color=hv.Cycle(list(color_mapping.values())), # Use 'glasbey' for 300 distinct colors
        line_width=1,
        width=800,
        height=400
    ),
    hv.opts.NdOverlay(
        show_legend=False, # 300 items would overlap the whole plot
        batched=True       # PERFORMANCE: This is critical for 300+ lines!
    )
)

plot

In [ ]:
hvplot.extension("bokeh")



ref_chart = df_ref.hvplot.points(x="G_ref", y="S_ref", color="wavelength_ref", cmap="spectral_r",
        size=12, colorbar=False, tools=["hover"])

sample_chart = df_samples.hvplot.points(x="G", y="S", color="sample", cmap=color_mapping,
        size=12, colorbar=False, tools=["hover", "box_select", "lasso_select", "tap"])



phasor_plot = (ref_chart * sample_chart).opts(
hv.opts.Points(
        frame_width=200,
        frame_height=200,
        size=12,
        padding=0.1,
        xlabel="G",
        ylabel="S",
        show_grid=True,
        title="Phasor Plot",
)
)

selection = streams.Selection1D(source=sample_chart)

hv.extension('bokeh')

from itertools import cycle
import colorcet
color_mapping = dict(zip(df_samples["sample"].to_list(), cycle(colorcet.b_glasbey_hv)))

hv_ds = hv.Dataset(ds.intensity.to_dataframe().reset_index(), kdims=['wavelength', 'sample'], vdims='intensity')

overlay = hv_ds.to(hv.Curve, 'wavelength', 'intensity').overlay('sample')

plot = overlay.opts(
    hv.opts.Curve(
        color=hv.Cycle(list(color_mapping.values())), 
        line_width=1,
        width=800,
        height=400
    ),
    hv.opts.NdOverlay(
        show_legend=False, 
        batched=True
    )
)



def select_spectrum(index):
    if not index:
        return plot
            
    else:
        return plot.select(sample=index)


phasor_plot + hv.DynamicMap(select_spectrum, streams=[selection])

In [ ]:
# TUTORIAL


# hv.extension('bokeh')

# # 1. Create your data/element
# points = hv.Points(np.random.rand(100, 2))

# # 2. Define the Selection1D stream linked to the points
# selection = hv.streams.Selection1D(source=points)

# # 3. Define a callback that uses the 'index' variable
# def callback(index):
#     # 'index' is a list of integer indices (e.g., [1, 5, 22])
#     if not index:
#         return hv.Text(0.5, 0.5, "No selection")
    
#     # Access the actual data using .iloc
#     selected_data = points.iloc[index]
#     mean_y = selected_data.dimension_values(1).mean()
    
#     return hv.Text(0.5, 0.5, f"Selected: {len(index)} points\nMean Y: {mean_y:.2f}")

# # 4. Link everything in a DynamicMap
# dmap = hv.DynamicMap(callback, streams=[selection])

# # Display both (ensure tools include selection tools)
# points.opts(tools=['tap', 'box_select', 'lasso_select'], size=10) + dmap

In [ ]:
hv.extension('bokeh')
hvplot.extension('bokeh')

import colorcet
len(colorcet.b_glasbey_hv)
#hv.Cycle.default_cycles['default_colors']=colorcet.


In [ ]:
len(unique_samples)

In [ ]:
from itertools import cycle
import colorcet

color_mapping = dict(zip(df_samples["sample"].to_list(), cycle(colorcet.b_glasbey_hv)))

ref_chart = df_ref.hvplot.points(
    x="G_ref", y="S_ref", color="wavelength_ref", cmap="spectral_r",
    size=12, colorbar=False, tools=["hover"]
)

sample_chart = df_samples.hvplot.points(
    x="G", y="S", 
    c="sample",          # Use 'c' or 'color'
    cmap=color_mapping,      # Passing dict ensures sample 67 is ALWAYS the same color
    size=12, colorbar=False, 
    tools=["box_select", "lasso_select", "tap", "hover"]
)

# 4. OPTIMIZED DYNAMICMAP
selection = streams.Selection1D(source=sample_chart)

# Pre-generate the full line plot once to avoid overhead in the callback
# This is a 'useful implementation' for performance: slice an existing plot
all_lines = ds.intensity.hvplot.line(
    x="wavelength",
    y="intensity",
    by="sample",
    cmap=color_mapping,      # Use the SAME dictionary here
    legend=False
)

def select_spectrum(index):
    if not index:
        # Return all lines but perhaps faded, or empty
        return all_lines
    
    return all_lines.select(sample=index)

# Compose
phasor_plot = (ref_chart * sample_chart).opts(
    hv.opts.Points(frame_width=300, frame_height=300, padding=0.1)
)

dmap = hv.DynamicMap(select_spectrum, streams=[selection])

phasor_plot + dmap